# Resnet : Optimization

In [1]:
import math
import numpy as np
import tensorflow as tf
import keras
from keras import applications, Model
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.metrics import make_scorer
from keras.utils.np_utils import to_categorical
import random
import sklearn
import pandas as pd
import warnings

warnings.simplefilter('ignore')

Using TensorFlow backend.


In [2]:
train_data_dir = 'Data/Training_Data'
validation_data_dir = 'Data/Validation_Data'
test_data_dir= 'Data/Test_Data'

training_features_file = 'Features/training_features_Resnet.npy'
validation_features_file = 'Features/validation_features_Resnet.npy'
top_weights_file = 'Weights/weights_Resnet.h5'
model_file = 'Models/model_Resnet.h5'

train_labels_file = 'Labels/training_labels.npy'
validation_labels_file = 'Labels/validation_labels.npy'
test_labels_file = 'Labels/test_labels.npy'

img_width, img_height = 224, 224
NB_CLASSES = 11

In [3]:
train_labels = np.load(open(train_labels_file, 'rb'))
validation_labels = np.load(open(validation_labels_file, 'rb'))
test_labels = np.load(open(test_labels_file, 'rb'))

In [4]:
print('Training Data : ' + str(len(train_labels)) + ' Images')
print('Validation Data : ' + str(len(validation_labels)) + ' Images')
print('Test Data : ' + str(len(test_labels)) + ' Images')

Training Data : 307 Images
Validation Data : 74 Images
Test Data : 18 Images


## Converting images to feature vectors using weights from ImageNet

In [5]:
def images_to_feature_vectors(model, directory, batch_size, steps):
    
    datagen = ImageDataGenerator()
    
    generator = datagen.flow_from_directory(
        directory,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) # Keep the data in the same order
    
    features = model.predict_generator(generator, steps, verbose=1) 
    
    return features

In [6]:
# Batch size has to be a multiple of the number of images  to keep our vectors consistents
training_batch_size = 1 # batch size for feature pre-training
validation_batch_size = 1 # batch size for feature pre-training

model = applications.ResNet50(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3))
training_features = images_to_feature_vectors(model, train_data_dir, training_batch_size, len(train_labels) // training_batch_size)
validation_features = images_to_feature_vectors(model, validation_data_dir, validation_batch_size, len(validation_labels) // validation_batch_size)

Instructions for updating:
Colocations handled automatically by placer.
Found 307 images belonging to 11 classes.
307/307 [==============================] - 35s 113ms/step
Found 74 images belonging to 11 classes.
74/74 [==============================] - 8s 112ms/step


In [7]:
with open(training_features_file, 'wb') as file:
        np.save(file, training_features, allow_pickle = False)
with open(validation_features_file, 'wb') as file:
        np.save(file, validation_features, allow_pickle = False)

## Randomized Search CV training

In [8]:
def create_model(lr, decay, nn1, nn2, nn3, input_shape, output_shape):
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture'''
    
    opt = keras.optimizers.Adam(lr=lr, decay=decay)
                                                     
    model = Sequential()
    
    model.add(Flatten())
    model.add(Dense(nn1, input_dim = input_shape, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(nn2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(nn3, activation='relu'))
    model.add(BatchNormalization())
            
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [9]:
model = KerasClassifier(build_fn=create_model, epochs=16, verbose=1)

In [10]:
# Learning rate values
lr=[1e-2, 1e-3, 1e-4]
decay=[1e-6,1e-9,0]

# Number of neurons per layer
nn1=[4096,2048,1024]
nn2=[2048,1024,512]
nn3=[1000,500,200]

batch_size=[2048,1024,512]

In [11]:
train_data = np.load(open(training_features_file, 'rb'))
#train_data = train_data.reshape(train_data.shape[0],-1)
validation_data = np.load(open(validation_features_file, 'rb'))
#validation_data = validation_data.reshape(validation_data.shape[0],-1)
    
train_labels_onehot = to_categorical(train_labels,NB_CLASSES)            #One Hot Encoder
validation_labels_onehot = to_categorical(validation_labels,NB_CLASSES)  #One Hot Encoder

In [12]:
# dictionary summary
param_grid = dict(
                    lr=lr, decay=decay, nn1=nn1, nn2=nn2, nn3=nn3,
                    batch_size=batch_size,
                    input_shape=train_data.shape[1:], output_shape = (NB_CLASSES,)
                 )


grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, 
                          verbose=20,  n_iter=10, n_jobs=1)

In [13]:
grid_result = grid.fit(train_data, train_labels_onehot)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] output_shape=101, nn3=500, nn2=512, nn1=4096, lr=0.0001, input_shape=2048, decay=0, batch_size=512 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Use tf.cast instead.
Epoch 1/16
2114/2114 [==============================] - 52s 24ms/step - loss: 3.4681 - acc: 0.3155
Epoch 2/16
2114/2114 [==============================] - 34s 16ms/step - loss: 0.1830 - acc: 0.9816
Epoch 3/16
2114/2114 [==============================] - 33s 16ms/step - loss: 0.0825 - acc: 0.9962
Epoch 4/16
2114/2114 [==============================] - 34s 16ms/step - loss: 0.0460 - acc: 0.9995
Epoch 5/16
2114/2114 [==============================] - 33s 16ms/step - loss: 0.0285 - acc: 1.0000
Epoch 6/16
2114/2114 [==============================] - 33s 16ms/step - loss: 0.0196 - acc: 1.0000
Epoch 7/16
2114/2114 [==============================] - 33s 15ms/step - loss: 0.0146 - acc: 1.0000
Epoch 8/16
2114/2114 [==============================] - 33s 16ms/step - loss: 0.0115 - acc: 1.0000
Epoch 9/16
2114/2114 [==============================] - 33s 16ms/step - loss: 0.0092 - acc: 1.0000
Epoch 10/16
2114/2114 [==============================] - 33s 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  9.3min remaining:    0.0s


Epoch 1/16
2114/2114 [==============================] - 52s 25ms/step - loss: 3.2600 - acc: 0.3642
Epoch 2/16
2114/2114 [==============================] - 35s 17ms/step - loss: 0.1710 - acc: 0.9820
Epoch 3/16
2114/2114 [==============================] - 35s 16ms/step - loss: 0.0717 - acc: 0.9957
Epoch 4/16
2114/2114 [==============================] - 34s 16ms/step - loss: 0.0396 - acc: 0.9995
Epoch 5/16
2114/2114 [==============================] - 34s 16ms/step - loss: 0.0251 - acc: 1.0000
Epoch 6/16
2114/2114 [==============================] - 35s 16ms/step - loss: 0.0174 - acc: 1.0000
Epoch 7/16
2114/2114 [==============================] - 35s 16ms/step - loss: 0.0129 - acc: 1.0000
Epoch 8/16
2114/2114 [==============================] - 35s 16ms/step - loss: 0.0102 - acc: 1.0000
Epoch 9/16
2114/2114 [==============================] - 35s 16ms/step - loss: 0.0083 - acc: 1.0000
Epoch 10/16
2114/2114 [==============================] - 34s 16ms/step - loss: 0.0071 - acc: 1.0000
Epoch 11/

...

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed: 85.3min remaining:    0.0s


Epoch 1/16
2114/2114 [==============================] - 30s 14ms/step - loss: 5.3447 - acc: 0.0241
Epoch 2/16
2114/2114 [==============================] - 14s 7ms/step - loss: 1.1702 - acc: 0.7569
Epoch 3/16
2114/2114 [==============================] - 14s 6ms/step - loss: 0.6108 - acc: 0.8765
Epoch 4/16
2114/2114 [==============================] - 14s 7ms/step - loss: 0.4662 - acc: 0.9101
Epoch 5/16
2114/2114 [==============================] - 14s 7ms/step - loss: 0.3661 - acc: 0.9338
Epoch 6/16
2114/2114 [==============================] - 14s 7ms/step - loss: 0.2820 - acc: 0.9371
Epoch 7/16
2114/2114 [==============================] - 14s 7ms/step - loss: 0.2503 - acc: 0.9385
Epoch 8/16
2114/2114 [==============================] - 14s 7ms/step - loss: 0.2145 - acc: 0.9508
Epoch 9/16
2114/2114 [==============================] - 14s 6ms/step - loss: 0.1514 - acc: 0.9664
Epoch 10/16
2114/2114 [==============================] - 14s 7ms/step - loss: 0.0982 - acc: 0.9816
Epoch 11/16
2114/2

2114/2114 [==============================] - 27s 13ms/step - loss: 0.0133 - acc: 1.0000
Epoch 15/16
2114/2114 [==============================] - 28s 13ms/step - loss: 0.0114 - acc: 1.0000
Epoch 16/16
1057/1057 [==============================] - 6s 6ms/step
[CV]  output_shape=101, nn3=500, nn2=1024, nn1=4096, lr=0.0001, input_shape=7, decay=0, batch_size=2048, score=0.003, total= 8.0min
[CV] output_shape=101, nn3=500, nn2=1024, nn1=4096, lr=0.0001, input_shape=7, decay=0, batch_size=2048 
Epoch 1/16
2114/2114 [==============================] - 59s 28ms/step - loss: 5.3519 - acc: 0.0260
Epoch 2/16
2114/2114 [==============================] - 28s 13ms/step - loss: 0.3198 - acc: 0.9565
Epoch 3/16
2114/2114 [==============================] - 28s 13ms/step - loss: 0.1662 - acc: 0.9839
Epoch 4/16
2114/2114 [==============================] - 27s 13ms/step - loss: 0.1029 - acc: 0.9957
Epoch 5/16
2114/2114 [==============================] - 28s 13ms/step - loss: 0.0702 - acc: 0.9995
Epoch 6/16
2

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 136.7min finished


Epoch 1/16
3171/3171 [==============================] - 29s 9ms/step - loss: 4.2111 - acc: 0.1583
Epoch 2/16
3171/3171 [==============================] - 14s 5ms/step - loss: 0.8028 - acc: 0.9313
Epoch 3/16
3171/3171 [==============================] - 14s 5ms/step - loss: 0.3614 - acc: 0.9845
Epoch 4/16
3171/3171 [==============================] - 14s 5ms/step - loss: 0.1857 - acc: 0.9981
Epoch 5/16
3171/3171 [==============================] - 14s 4ms/step - loss: 0.1124 - acc: 1.0000
Epoch 6/16
3171/3171 [==============================] - 14s 4ms/step - loss: 0.0762 - acc: 1.0000
Epoch 7/16
3171/3171 [==============================] - 14s 4ms/step - loss: 0.0550 - acc: 1.0000
Epoch 8/16
3171/3171 [==============================] - 14s 4ms/step - loss: 0.0421 - acc: 1.0000
Epoch 9/16
3171/3171 [==============================] - 14s 4ms/step - loss: 0.0335 - acc: 1.0000
Epoch 10/16
3171/3171 [==============================] - 14s 4ms/step - loss: 0.0277 - acc: 1.0000
Epoch 11/16
3171/31

In [14]:
cv_results_df = pd.DataFrame(grid_result.cv_results_)
cv_results_df.to_csv('gridsearch_Inception.csv')

In [15]:
cv_results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_output_shape,param_nn3,param_nn2,param_nn1,param_lr,param_input_shape,param_decay,param_batch_size,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,565.142282,7.540171,4.024790,0.160267,101,500,512,4096,0.0001,2048,0,512,"{'output_shape': 101, 'nn3': 500, 'nn2': 512, ...",0.000946,0.000946,0.003784,0.001892,0.001338,8
1,99.752007,0.786410,2.097494,0.036809,101,200,512,1024,0.01,7,0,2048,"{'output_shape': 101, 'nn3': 200, 'nn2': 512, ...",0.000000,0.001892,0.002838,0.001577,0.001180,10
2,300.735591,0.652292,3.001509,0.103047,101,1000,1024,2048,0.0001,7,1e-09,512,"{'output_shape': 101, 'nn3': 1000, 'nn2': 1024...",0.003784,0.000000,0.003784,0.002523,0.001784,4
3,220.918611,0.898483,3.462500,0.448275,101,500,2048,2048,0.001,2048,1e-09,1024,"{'output_shape': 101, 'nn3': 500, 'nn2': 2048,...",0.000946,0.000000,0.003784,0.001577,0.001608,9
4,302.510548,3.925874,4.025182,0.670291,101,1000,512,2048,0.0001,2048,1e-06,512,"{'output_shape': 101, 'nn3': 1000, 'nn2': 512,...",0.003784,0.003784,0.005676,0.004415,0.000892,2
5,128.716579,0.950203,3.549498,0.371872,101,200,2048,1024,0.0001,7,0,1024,"{'output_shape': 101, 'nn3': 200, 'nn2': 2048,...",0.006623,0.002838,0.006623,0.005361,0.001784,1
6,227.970891,20.574100,4.292315,0.274153,101,500,2048,2048,0.001,2048,1e-09,2048,"{'output_shape': 101, 'nn3': 500, 'nn2': 2048,...",0.002838,0.000000,0.003784,0.002208,0.001608,6
7,180.298902,1.525775,4.206313,0.230874,101,1000,512,1024,0.001,7,0,1024,"{'output_shape': 101, 'nn3': 1000, 'nn2': 512,...",0.003784,0.000000,0.003784,0.002523,0.001784,4
8,186.447254,0.599672,4.549720,0.076046,101,200,2048,1024,0.001,2048,1e-06,1024,"{'output_shape': 101, 'nn3': 200, 'nn2': 2048,...",0.004730,0.000946,0.003784,0.003154,0.001608,3
9,481.641443,3.385018,6.589171,0.435379,101,500,1024,4096,0.0001,7,0,2048,"{'output_shape': 101, 'nn3': 500, 'nn2': 1024,...",0.002838,0.000000,0.003784,0.002208,0.001608,6


In [16]:
grid_result.best_params_

{'output_shape': 101,
 'nn3': 200,
 'nn2': 2048,
 'nn1': 1024,
 'lr': 0.0001,
 'input_shape': 7,
 'decay': 0,
 'batch_size': 1024}

In [13]:
# Parameters after RandomizedSearchCV

nn1 = 1024; nn2 = 1024; nn3 = 200
lr = 0.0001; decay=0
batch_size = 1024

## Regularization

In [14]:
# Regularzation Parameters

dropout = 0.5
l1 = 0.0001
l2 = 0.0001

In [15]:
def model():
    opt = keras.optimizers.Adam(lr=lr)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    
    model.add(Flatten())
    model.add(Dense(nn1, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(nn2, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(nn3, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
            
    model.add(Dense(NB_CLASSES, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [16]:
train_labels_onehot = to_categorical(train_labels, NB_CLASSES)            #One Hot Encoder
validation_labels_onehot = to_categorical(validation_labels, NB_CLASSES)  #One Hot Encoder

In [17]:
top = model()

In [18]:
top.fit(train_data, train_labels_onehot,
              epochs=32,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels_onehot))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 307 samples, validate on 74 samples
Epoch 1/32
307/307 [==============================] - 26s 85ms/step - loss: 46.4672 - acc: 0.0879 - val_loss: 45.0449 - val_acc: 0.5405
Epoch 2/32
307/307 [==============================] - 3s 11ms/step - loss: 45.6175 - acc: 0.2248 - val_loss: 44.4285 - val_acc: 0.7838
Epoch 3/32
307/307 [==============================] - 3s 11ms/step - loss: 44.8443 - acc: 0.4397 - val_loss: 44.0006 - val_acc: 0.8649
Epoch 4/32
307/307 [==============================] - 3s 11ms/step - loss: 44.4915 - acc: 0.4495 - val_loss: 43.6444 - val_acc: 0.9054
Epoch 5/32
307/307 [==============================] - 3s 11ms/step - loss: 44.0419 - acc: 0.5635 - val_loss: 43.3404 - val_acc: 0.9054
Epoch 6/32
307/307 [==============================] - 4s 11ms/step - loss: 43.7606 - acc: 0.6352 - val_loss: 43.0652

In [19]:
top.save_weights(top_weights_file)

## Complete Model

In [20]:
def complete_model():
    opt = keras.optimizers.Adam(lr=lr)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                 
    base = applications.ResNet50(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3))
    
    top = Sequential()

    top.add(Flatten(input_shape=train_data.shape[1:]))
    top.add(Dense(nn1, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())
    top.add(Dense(nn2, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())
    top.add(Dense(nn3, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())            
    top.add(Dense(NB_CLASSES, activation='softmax'))
    top.load_weights(top_weights_file)
    top.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    
    
    model = Model(input= base.input, output= top(base.output))
    for layer in model.layers:
        layer.trainable = False
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [21]:
model = complete_model()

datagen = ImageDataGenerator()

print("Model Summary")
print(model.summary())

Model Summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
_______________________________________________________________________________________________

In [22]:
generator = datagen.flow_from_directory(
            test_data_dir,
            target_size=(img_width, img_height),
            batch_size=len(test_labels),
            class_mode=None,
            shuffle=False)
test_predictions = model.predict_generator(generator, 1, verbose=1) 

Found 18 images belonging to 11 classes.
1/1 [==============================] - 4s 4s/step


In [23]:
test_predictions = np.asarray(list(map(str,np.argmax(test_predictions,axis=1)))).reshape(-1,1)
test_labels = np.asarray(test_labels).reshape(-1,1)
test_acc, test_acc_op = tf.metrics.accuracy(test_labels, test_predictions)

Instructions for updating:
Use tf.cast instead.


In [24]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [25]:
test_accuracy = sess.run(test_acc_op)

In [26]:
print('\nTest accuracy : ' + str(test_accuracy))


Test accuracy : 0.9444444


In [27]:
model.save(model_file)